# Category_Classify
## Goal of the function
* The dataSource of the func is M-List
* Classify the material in the M-List, and fill the new colum "Category" with string such as "Fabric", "Zipper", "Button" etc.
* Save the result as a Categoried_M_List

In [53]:
import pandas as pd
import numpy as np

import nltk
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# File system
from os import walk
from os.path import join

# Switch area
CATEGORY_LIST =['fabric', 'insulation', 'interfacing', 'zipper', 'label']

# SUBJECT = 'label'
VOCAB_SIZE = 900

In [54]:
FROM_DIR = 'DataSource/M-List'
TO_DIR = 'result'

In [55]:
#  M_List = pd.read_csv(FROM_DIR + '/AB18MJ1_032_ BOM_032 BOM_material_list.csv', encoding = 'ISO-8859-1')

* df.insert(1, 'Category', '', True)

* df.at[0,'Category'] = 'Steve'

# Functions
* We need to convert the M_list(Type in DF)through several process to Full matrix, so we can classify the M_List with the trained_data we made before.
## Process
### delete_col()
### stemmered_nltk_convert()
### turn_series()
### make_sparse_matrix()
### make_full_feature()

## delete_col()
* Delete the none columns 

In [56]:
def delete_col(m_list):
    for col in m_list:
        if m_list[col].count() == 0:
            m_list = m_list.drop(col, axis = 1)
        
    return m_list

## stemmered_nltk_convert() 
* Nltk stemmered Function

In [57]:
def stemmered_nltk_convert(col_of_df):
    '''
    Parameter of this function is a column of a dataFrame.
    
    '''
    # difine Stop words
    stop_words = set(stopwords.words('english'))
    # Difine Stemmer
    stemmer = SnowballStemmer('english')
    
    # converts to lower case and splits up the words
    words = word_tokenize(col_of_df)
    filtered_words = []
    
    for word in words:
        # Removes the stop words and punctuation
        # if word is not in the stop_words list and is not a alpha.
        if word not in stop_words and word.isalpha():
            filtered_words.append(stemmer.stem(word))
            
    return filtered_words

## turn_series()
#### Walk through bom
* Parse a xlsm of bom, turn the row into a cell, all the cells will form a col.
* Put the cell to the classify function
* Return the index of row that is True

In [58]:
def turn_series(bom):
    '''
    
    The parameter of the func is a dataFrame
    
    '''
    database = []
    
    for row in bom.index:
        row_str = str()
        for col in bom:
            row_str = row_str + ', ' + str(bom.at[row, col])
        database.append(row_str)
    
    col = pd.Series(database)
#     index_list = classify_series(col)
    
    return col

## make_sparse_matrix()
### Sparse Matrix Function
* Create a sparse Matrix for the data we want to predict
* The difference of this function in comparition with Classification Model for Train data, is this function don't need CATEGORY.

In [59]:
def make_sparse_matrix(df, vocabulary):
    """
    Param1:
    The data we want to sparse, which must be in format of DataFrame.
    
    Param2:
    The vocabulary, it is generated when we training datas.
    
    Returns a sparse matrix as dataframe
    """
 
    indexed_words = pd.Index(vocabulary.VOCAB_WORD)
    nr_rows = df.shape[0]
    nr_cols = df.shape[1]
    word_set = set(indexed_words)
    dict_list = []
    
    for i in range(nr_rows):
        for j in range(nr_cols):
            
            word = df.iat[i, j]
            if word in word_set:
                doc_id = df.index[i]
                word_id = indexed_words.get_loc(word)
                
                item = {'MATERIAL_ID': doc_id,
                       'OCCURENCE': 1, 'WORD_ID': word_id}
                
                dict_list.append(item)
                
    return pd.DataFrame(dict_list)

## make_full_feature()
### Full Matrix
* Since we want to predict the data, so we create the Full Feature directly.

In [60]:
def make_full_feature(sparse_matrix, nr_words, doc_idx = 0, word_idx = 1, freq_idx = 2):
    column_names = ['MATERIAL_ID'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index = doc_id_names, columns = column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'MATERIAL_ID'] = doc_nr
        full_matrix.at[doc_nr, word_id] = occurrence
        
    full_matrix.set_index('MATERIAL_ID', inplace = True)
    return full_matrix

# Core Funciton
## Material_Classifor
* Classifor the materials in the M-List, 
* Mateiral is in row direction in a M-List.
* Return a list of index of row that classified as "true"

In [61]:
def material_classifor(bom, SUBJECT):
    VOCAB  = 'DataSource/Trained Data/' + SUBJECT + '_vocabulary.csv'
    TRAIN_DATA_1 = 'DataSource/Trained Data/' + SUBJECT + '_prob_tokens_ctg_1_in_train_data'
    TRAIN_DATA_0 = 'DataSource/Trained Data/' + SUBJECT + '_prob_tokens_ctg_0_in_train_data'
    TRAIN_DATA_ALL = 'DataSource/Trained Data/' + SUBJECT + '_prob_tokens_all_in_train_data'
    PROB_1_TRAIN_DATA = 'DataSource/Trained Data/' + SUBJECT + '_prob_ctg_1_in_train_data'
    '''
    Param_1
    <bom>
    DataFrame, by reading M-List in csv format.
    
    Param_2
    <SUBJECT>
    It defines what to analyze, for example, of the SUBJECT == 'fabric', 
    then the func will use the trained_data set of fabric to analyze the documents.
    
    Local_var_1
    <VOCAB>
    The path of the vocabulary
    Token list with WORD_ID
    
    Local_var_2
    <TRAIN_DATA_1>
    The trained data of catagory True
    Probabilitie of each token in category True
    
    Local_var_3
    <TRAIN_DATA_0>
    The trained data of category False
    Probabilitie of each token in category False
    
    Local_var_4
    <TRAIN_DATA_ALL>
    The trained data of category both.
    Probabilitie of each token in all documents
    
    Local_var_5
    <PROB_1_TRAIN_DATA>
    The percentage of documents in catagory True in all documents.
    Number of documents in catagory True / number of all documents
    '''
    # read the vocabulary
    vocab = pd.read_csv(VOCAB, index_col = 0)
    # read the trained_datas
    train_data_1 = np.loadtxt(TRAIN_DATA_1)
    train_data_0 = np.loadtxt(TRAIN_DATA_0)
    train_data_all = np.loadtxt(TRAIN_DATA_ALL)
    prob_ctg_1 = pd.read_csv(PROB_1_TRAIN_DATA, index_col = 0)
    prob_ctg_1_train_data = prob_ctg_1.loc[0, 'prob_ctg_1_train_set']
    
    
    # Delete useless cols
    col_deleted_bom = delete_col(bom)
    
    # Series
    # Parse the bom, make each col getting together to be 1 col
    new_bom = turn_series(col_deleted_bom)
    
    # nltk_convert
    stemmed_bom = new_bom.apply(stemmered_nltk_convert)
    
    # Convert the stemmed series into df
    # 1 token get 1 cell
    word_col_df = pd.DataFrame.from_records(stemmed_bom.tolist())
    
    # Sparse Matrix
    # Create a sparse Matrix for the data we want to predict
    # The difference of this function in comparition with Classification Model for Train data, is this function don't need CATEGORY.
    sparse_predict_df = make_sparse_matrix(word_col_df, vocab)
    # Grouped by MATERIAL_ID
    sparse_predict_df_grouped = sparse_predict_df.groupby(['MATERIAL_ID', 'WORD_ID']).sum()
    # Reset it index
    sparse_predict_df_grouped = sparse_predict_df_grouped.reset_index()
    # Convert it into numpy array.
    sparse_predict_data = sparse_predict_df_grouped.to_numpy()
    
    #Full Matrix
    predict_full_feature = make_full_feature(sparse_predict_data, vocab.shape[0])
    
    #Joint probability in log format
    joint_log_ctg_1 = predict_full_feature.dot(np.log(train_data_1) - np.log(train_data_all)) + np.log(prob_ctg_1_train_data)
    joint_log_ctg_0 = predict_full_feature.dot(np.log(train_data_0)-np.log(train_data_all))+np.log(1 - prob_ctg_1_train_data)
    # Prediction
    prediction_log = joint_log_ctg_1 > joint_log_ctg_0
    
    # Get the index of the row that predicted as material in the bom
    row_list = prediction_log[prediction_log == True].index

    #2020/03/11
    # The difference with the func "M-List_generator" is that the func only return a list of index classified as True.
    # Later I may optimize the func "M-List_generator" same as this func, so the two func can use same code as this func.
    # Let the different part be done outside the func.
#     # Get the material from the original bom by the index in row_list
#     material_list = bom.loc[row_list,:]
    
    return row_list

# Main Funciton
## Loop through materials designated as category.
* Analyze a M-List with several sets of train-data and vocabulary. 
* Each set of trainned-data and vocabulary represents 1 category of material, such as fabric, zipper, label.
* This func will feed the func "material_classifor" each set of trainned-data and vocabulary by order.

In [62]:
M = 'DataSource/M-List/AB18MJ1_032_ BOM_032 BOM_material_list.csv'

In [63]:
def fill_out_col_category(M_list, Style_name, LIST = CATEGORY_LIST) :
    '''
    Param_1
    <M_list>
    M_list in CSV format
    The func will turn it into a DataFrame while calculating.
    
    Param_2
    The name of the file.

    Param_3
    An array in type List.
    The content is the categories of material.
    That decides which train_set data to be used.
    '''
    bom = pd.read_csv(M_list, index_col = None, encoding = 'ISO-8859-1')
    bom.insert(1, 'CATEGORY', 'other', True)
    
    for cate in LIST:
        ROW_LIST = material_classifor(bom, cate)
        bom.at[ROW_LIST, 'CATEGORY'] = cate
    
    bom.to_csv('result/' + Style_name + '_CATE_classified_M-List.csv')
    return bom

## Walk through a dir 
* Check each xlsx in a dir

In [64]:
def m_list_cate_classify_convertor(FROM_DIR, TO_DIR):
    converted_csv_num = 0
    for root, dirnames, filenames in walk(FROM_DIR):
        # walk through each xlsx file
        for file_name in filenames:
             # get the path of the file
            # Appoint the method only work with .xlsx file.
            if file_name.endswith('.csv') :
                converted_csv_num = converted_csv_num + 1
                filepath = join(root, file_name)
                # Custom function
                fill_out_col_category(filepath, file_name[0:-18])
    print('Converted ', converted_csv_num, ' files')

In [65]:
%%time
m_list_cate_classify_convertor(FROM_DIR, TO_DIR)

Converted  2  files
CPU times: user 3.57 s, sys: 41.8 ms, total: 3.62 s
Wall time: 2.01 s
